In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager as chrome
import pandas as pd
import time
import re

In [2]:
# Initialize the chrome browser in splinter
executable_path={"executable_path":chrome().install()}
browser=Browser("chrome",**executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\lol\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# NASA Mars News

**Objetive:** Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later

In [3]:
# Visit the mars nasa news site
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)

nasa_html = browser.html
soup = bs(nasa_html, "html.parser")

In [4]:
# Scrape NASA site to find container of news titles with date, title and description
news_list = soup.find("div", class_="list_text")
# print(news_list)

#First news title
news_title = news_list.find("a").text
print(news_title)

# Description of news title (paragraph)
news_paragraph = news_list.find("div", class_="article_teaser_body").text
print(news_paragraph)


NASA Ingenuity Mars Helicopter Prepares for First Flight
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


# JPL Mars Space Images - Featured Image
**Objetive:** Visit the url for JPL Featured Space Image and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url

In [5]:
# JPL url
jpl_url = "https://www.jpl.nasa.gov"

# Search for Mars images
search = "/images?search=&category=Mars"

# Visit the site with chrome 
browser.visit(jpl_url+search)

# Parse the html (Mars Image Gallery)
jpl_html = browser.html
jpl_soup = bs(jpl_html, "html.parser")

In [6]:
# Find the lastest Mars image link (first link on the list)
image_link = jpl_soup.find("div", class_="SearchResultCard").find("a", class_="group")
image_link = jpl_url + image_link['href']
print(image_link)

https://www.jpl.nasa.gov/images/namibia-dunes


In [7]:
# Visit the Mars image link
browser.visit(image_link)

# Parse the new web page
mars_image_html = browser.html
images_soup = bs(mars_image_html, "html.parser")

In [8]:
# Find the link to the original image (the biggest one 3327 x 1677 pixels)
featured_image_url = images_soup.find("div", class_="PageImageDetail").find("meta")["content"]
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24551.jpg


# Mars Facts
**Objetive:** Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet. Use Pandas to convert the data to a HTML table string.

In [9]:
# Read the html and convert to a Data Frame
facts_df = pd.read_html('https://space-facts.com/mars/')[0]
# Append columns
facts_df.columns = ['description', 'value']
# Set index
facts_df.set_index('description', inplace=True)
# Display data frame
facts_df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
# Convert data frame to html
facts_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

# Mars Hemispheres
**Objetive:** Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres. Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name.

In [11]:
# USGS Astrogeology web site URL
astro_url = "https://astrogeology.usgs.gov"
search = "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(astro_url + search)
time.sleep(5)
astro_html = browser.html
astro_soup = bs(astro_html, "html.parser")

In [12]:
# Hemisphere data contained in items 
items = astro_soup.find_all("div", class_ = "item")

# Hemisphere list
hemisphere_image_urls=[]

# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name.
# Use a Python dictionary to store the data using the keys img_url and title.
for item in items:
    
    title = item.find("h3").text
    image_url = item.find("a", class_ = "itemLink product-item")["href"]
    
    # Visit the new link for full image  
    browser.visit(astro_url + image_url)
    
    # Parse html
    image_html = browser.html
    soup = bs(image_html, "html.parser")
    
    # Image URL
    image_url = astro_url + soup.find("img", class_ = "wide-image")["src"]
    
    # Append to a Python dictionary
    hemisphere_image_urls.append({"title" : title, "img_url" : image_url})

# Display the list
hemisphere_image_urls
  

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [13]:
browser.quit()